In [48]:
import torch 
import torch.nn as nn

import numpy as np
import math
from collections import OrderedDict

In [49]:
# MLP
class NN(nn.Module):
    """
    A simple Multi-Layer Perceptron (MLP) implemented using PyTorch.
    
    Arguments:
    ----------
    in_size : int
        The size of the input layer (number of features in the input).
    h_size : int
        The size of the hidden layers (number of neurons in each hidden layer).
    o_size : int
        The size of the output layer (number of classes or target values).
    n_layers : int, optional (default=5)
        The number of hidden layers in the network.
    
    Methods:
    --------
    forward(x):
        Defines the forward pass of the network, applying the layers sequentially to the input.
        
    Example:
    --------
    >>> model = NN(in_size=10, h_size=20, o_size=5, n_layers=3)
    >>> x = torch.randn(1, 10)  # A batch of 1 with 10 input features
    >>> output = model(x)
    """
    def __init__(self, in_size, h_size, o_size, n_layers=5):
        super(NN, self).__init__()
        # Create layers pipeline
        layers = []
        # Establish the non-linear activation function
        activation = nn.Tanh
        # Add input in the pipeline & the activation
        layers.append(("in", nn.Linear(in_size, h_size)))
        layers.append(("in_activation", activation()))
        # Add the hidden layers
        for i in range(n_layers):
            layers.append(
                ("layer_%d" %i, nn.Linear(h_size, h_size))
            )
            layers.append(("activation_layer_%d" %i, activation()))
        layers.append(("output", nn.Linear(h_size, o_size)))
        layer_dict = OrderedDict(layers)
        self.layers = nn.Sequential(layer_dict)
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [50]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
step_x = 0.1
step_t = 0.1
x = torch.arange(-1, 1 + step_x, step_x)
t = torch.arange(0, 1 + step_t, step_t)

# Grid: Exact Solution
X = torch.stack(torch.meshgrid(x, t)).reshape(2, -1).T.to(device)
X.requires_grad = True

# training data
boundary_condition_start = torch.stack(torch.meshgrid(x[0], t)).reshape(2, -1).T
boundary_condition_end = torch.stack(torch.meshgrid(x[-1], t)).reshape(2, -1).T
initial_condition = torch.stack(torch.meshgrid(x, t[0])).reshape(2, -1).T
x_train = torch.cat([boundary_condition_start, boundary_condition_end, initial_condition]).to(device)
y_bound_cond_start = torch.zeros(len(boundary_condition_start))
y_bound_cond_end = torch.zeros(len(boundary_condition_end))
y_init_cond = -torch.sin(math.pi * initial_condition[:,0])
y_train = torch.cat([y_bound_cond_start, y_bound_cond_end, y_init_cond]).unsqueeze(1).to(device)


# Initialize model
model = NN(in_size=2, h_size=20, o_size=1, n_layers=4).to(device)
criterion = nn.MSELoss()
iter = 1 #???

optimizer = torch.optim.LBFGS(
    model.parameters(),
    lr=0.1,
    max_iter=50000,
    max_eval=50000,
    history_size=50,
    tolerance_grad=1e-7,
    tolerance_change=1.0*np.finfo(float).eps,
    line_search_fn="strong_wolfe"
)

adam = torch.optim.Adam(model.parameters())

In [96]:
def loss_function(model, X, X_train, y_train, criterion):
    y_pred = model(X_train)
    loss_data = criterion(y_pred, y_train)
    u = model(X)

    du_dX = torch.autograd.grad(
        inputs = X,
        outputs= u,
        grad_outputs = torch.ones_like(u),
        retain_graph = True,
        create_graph = True
    )[0]

    du_dt = du_dX[:, 1]
    du_dx = du_dX[:, 0]

    du_dxx = torch.autograd.grad(
        inputs = X,
        outputs= du_dX,
        grad_outputs = torch.ones_like(du_dX),
        retain_graph = True,
        create_graph = True
    )[0][:,0]
    loss_pde = criterion(du_dt + u.squeeze()*du_dx, 0.01/math.pi * du_dxx)

    loss = loss_pde + loss_data
    loss.backward()
    return loss

In [95]:
# Training with Adam optimizer
EPOCHS = 100
for epoch in range(EPOCHS):
    # Zero the gradients
    adam.zero_grad()
    
    # Compute the loss using the custom loss function
    loss = loss_function(model, X, x_train, y_train, criterion)
    
    # Perform a step with the Adam optimizer
    adam.step()  # Update model parameters

    # Optional: Print loss at intervals for debugging purposes
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.6f}")

# Fine-tuning with L-BFGS optimizer
def closure():
    # Zero the gradients
    optimizer.zero_grad()
    
    # Compute the loss
    loss = loss_function(model, X, x_train, y_train, criterion)
    print(f"L-BFGS Loss: {loss.item():.6f}")
    return loss

# Apply L-BFGS after training loop
optimizer.step(closure)

Epoch 0: Loss = 0.000057
Final loss after L-BFGS: 0.125605
Final loss after L-BFGS: 0.125494
Final loss after L-BFGS: 0.125029
Final loss after L-BFGS: 0.124453
Final loss after L-BFGS: 0.124284
Final loss after L-BFGS: 0.123777
Final loss after L-BFGS: 0.122600
Final loss after L-BFGS: 0.121523
Final loss after L-BFGS: 0.118729
Final loss after L-BFGS: 0.117840
Final loss after L-BFGS: 0.116386
Final loss after L-BFGS: 0.111778
Final loss after L-BFGS: 0.110582
Final loss after L-BFGS: 0.109515
Final loss after L-BFGS: 0.104604
Final loss after L-BFGS: 0.102920
Final loss after L-BFGS: 0.101340
Final loss after L-BFGS: 0.099578
Final loss after L-BFGS: 0.098193
Final loss after L-BFGS: 0.096702
Final loss after L-BFGS: 0.095180
Final loss after L-BFGS: 0.093380
Final loss after L-BFGS: 0.092379
Final loss after L-BFGS: 0.088643
Final loss after L-BFGS: 0.088026
Final loss after L-BFGS: 0.087360
Final loss after L-BFGS: 0.084587
Final loss after L-BFGS: 0.083676
Final loss after L-BFGS

KeyboardInterrupt: 

In [89]:
# Multi-layer Perceptron
class NN(nn.Module):
    def __init__(
        self,
        input_size,
        hidden_size,
        output_size,
        depth,
        act=torch.nn.Tanh,
    ):
        super(NN, self).__init__()
        
        layers = [('input', torch.nn.Linear(input_size, hidden_size))]
        layers.append(('input_activation', act()))
        for i in range(depth): 
            layers.append(
                ('hidden_%d' % i, torch.nn.Linear(hidden_size, hidden_size))
            )
            layers.append(('activation_%d' % i, act()))
        layers.append(('output', torch.nn.Linear(hidden_size, output_size)))

        layerDict = OrderedDict(layers)
        self.layers = torch.nn.Sequential(layerDict)

    def forward(self, x):
        out = self.layers(x)
        return out

class Net:
    def __init__(self):
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

        self.model = NN(
            input_size=2,
            hidden_size=20,
            output_size=1,
            depth=4,
            act=torch.nn.Tanh
        ).to(device)
        
        self.h = 0.1
        self.k = 0.1
        x = torch.arange(-1, 1 + self.h, self.h)
        t = torch.arange(0, 1 + self.k, self.k)

        # exact solution
        self.X = torch.stack(torch.meshgrid(x, t)).reshape(2, -1).T
        
        # training data
        bc1 = torch.stack(torch.meshgrid(x[0], t)).reshape(2, -1).T
        bc2 = torch.stack(torch.meshgrid(x[-1], t)).reshape(2, -1).T
        ic = torch.stack(torch.meshgrid(x, t[0])).reshape(2, -1).T
        self.X_train = torch.cat([bc1, bc2, ic])
        y_bc1 = torch.zeros(len(bc1))
        y_bc2 = torch.zeros(len(bc2))
        y_ic = -torch.sin(math.pi * ic[:, 0])
        self.y_train = torch.cat([y_bc1, y_bc2, y_ic])
        self.y_train = self.y_train.unsqueeze(1)
        
        self.X = self.X.to(device)
        self.X_train = self.X_train.to(device)
        self.y_train = self.y_train.to(device)
        self.X.requires_grad = True
        
        self.criterion = torch.nn.MSELoss()
        self.iter = 1
        
        self.optimizer = torch.optim.LBFGS(
            self.model.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-7, 
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe",   # better numerical stability
        )
        
        self.adam = torch.optim.Adam(self.model.parameters())
        
    def loss_func(self):
        # this is more like a not so elegant hack to zero grad both optimizers
        self.adam.zero_grad()
        self.optimizer.zero_grad()
        
        y_pred = self.model(self.X_train)
        loss_data = self.criterion(y_pred, self.y_train)
        u = self.model(self.X)

        du_dX = torch.autograd.grad(
            inputs=self.X, 
            outputs=u, 
            grad_outputs=torch.ones_like(u), 
            retain_graph=True, 
            create_graph=True
        )[0]
        
        du_dt = du_dX[:, 1]
        du_dx = du_dX[:, 0]
        du_dxx = torch.autograd.grad(
            inputs=self.X, 
            outputs=du_dX, 
            grad_outputs=torch.ones_like(du_dX), 
            retain_graph=True, 
            create_graph=True
        )[0][:, 0]
        
        loss_pde = self.criterion(du_dt + u.squeeze() * du_dx, 0.01 / math.pi * du_dxx)

        loss = loss_pde + loss_data
        loss.backward()
        if self.iter % 100 == 0: 
            print(self.iter, loss.item())
        self.iter = self.iter + 1
        return loss
    
    def train(self):
        self.model.train()
        for i in range(10):
            self.adam.step(self.loss_func)
        self.optimizer.step(self.loss_func)
        
    def eval_(self):
        self.model.eval()

In [90]:
net = Net()
net.train()

100 0.04124385491013527
200 0.012515479698777199
300 0.007601000368595123
400 0.004934549331665039
500 0.003005249658599496
600 0.001541044213809073
700 0.0012362673878669739
800 0.0009917817078530788
900 0.0007686689496040344
1000 0.000608782924246043
1100 0.0005187066271901131
1200 0.0004733972600661218
1300 0.000423105841036886
1400 0.00039169981027953327
1500 0.00035269546788185835
1600 0.00031349260825663805
1700 0.0002836181374732405
1800 0.00026387395337224007
1900 0.00024768474395386875
2000 0.00023101689293980598
2100 0.0002141190634574741
2200 0.00018740579253062606
2300 0.00017095525981858373
